### IMU予積分(IMU Pre-integration)
IMU予積分(IMU Pre-integration)はVisual Odometry or SLAMでよく使われた手法です、フレーム間のIMU情報を制約として最適化フレームワークに導入することで、より高精度なOdometry or SLAM
を実現できる。しかし、IMU予積分予積分の式はすごく難しい、どうやって導出しているかどうかについて論文から説明がありません、この文章の目的はIMU予積分の式を導出します。

### IMUのモデル
Gyroモデル
$$\tilde{\omega}_b= \omega_b + bg + ng\tag{1}$$

* $\tilde{\omega}_b$はGyroの測量値, 下付き文字はbはimuのbody座標計を示す
* $\omega_b$はGyroの真値
* $bg$はGyroのbias
* $ng$はGyroのnoise

加速度計モデル
$$\tilde{a}_b= q_{bw}(a_w+g_w) + ba + na\tag{2}$$
* $\tilde{a}_b$は加速度の測量値, 下付き文字はbはimuのbody座標計を示す
* $q_bw$世界座標系をbody座標系に変換するためのクオータニオン
* $a_w$は加速度の真値(世界座標系)
* $g_w$は重量加速度(世界座標系)
* $ba$は加速度のbias
* $na$は加速度のnoise


### 連続時間でIMUの運動モデル
i時刻からj時刻まで
位置(position)：
$$p_{wb_{j}}= p_{wb_{i}} + v^w_i\Delta t + \int\int_{t\in [i,j]}(q_{wb_{t}}a^{b_t}-g^w)\delta t^2 $$
$$p_{wb_{j}}= p_{wb_{i}} + v^w_i\Delta t - \frac{1}{2} g^w \Delta t^2 + \int\int_{t\in [i,j]}(q_{wb_{t}}a^{b_t})\delta t^2 $$
$$p_{wb_{j}}= p_{wb_{i}} + v^w_i\Delta t - \frac{1}{2} g^w \Delta t^2 + q_{wb_{i}} \int\int_{t\in [i,j]}(q_{b_i b_t}a^{b_t})\delta t^2 \tag{3} $$


速度(velocity)：
$$v_j^w= v_i^w + \int_{t\in [i,j]}(q_{wb_{t}}a^{b_t}-g^w)\delta t$$
$$v_j^w= v_i^w + g^w\Delta t + \int_{t\in [i,j]}(q_{wb_{t}}a^{b_t})\delta t $$
$$v_j^w= v_i^w + g^w\Delta t + + q_{wb_{i}} \int_{t\in [i,j]}(q_{b_{i} b_{t}}a^{b_t})\delta t \tag{4} $$


姿勢(quaternion)：
$$q_{wb_{j}}= \int_{t\in [i,j]} q_{wb_{t}} \otimes \left[
\begin{matrix} 0 \\ \omega^{b_t}/2 \end{matrix}
\right] \delta t \tag{5} $$
  
  
(3)(4)(5)で、以下の分量はIMUの測量値のみ関係するため、予積分量と呼びます
$$\alpha_{b_i b_j} =\int\int_{t\in [i,j]}(q_{b_i b_t}a^{b_t})\delta t^2 $$
$$\beta_{b_i b_j} =\int_{t\in [i,j]}(q_{b_{i} b_{t}}a^{b_t})\delta t $$
$$q_{b_i b_j} =\int_{t\in [i,j]} q_{wb_{t}} \otimes \left[
\begin{matrix} 0 \\ \omega^{b_t}/2 \end{matrix}
\right] \delta t $$




### IMUの運動モデルの離散化表現
$$p_{wb_{k+1}}= p_{wb_{k}} + v^w_{b_k}\Delta t + \frac{1}{2} \hat{a} \Delta t^2 $$
$$q_{w b_{k+1}} = q_{wb_{t}} \otimes \left[
\begin{matrix} 1 \\ \hat{\omega} \delta t /2 \end{matrix}
\right] $$
$$v^w_{k+1}= v_k^w + \hat{a}\Delta t $$
$$b^{a}_{k+1} = b^{a}_{k} + n_{b^{a}_{k}}\Delta t $$
$$b^{g}_{k+1} = b^{g}_{k} + n_{b^{g}_{k}}\Delta t $$


here:
 $$\hat{a} = \frac{1}{2} [ q_{wb_{k}}(a^{b_k} - b^a_k) - g^w + q_{w b_{k+1}}(a^{b_{k+1}} -b^a_k)-g^w ]$$
 $$\hat{\omega} = \frac{1}{2} [ (\omega^{b_k} - b^g_k) + (\omega^{b_{k+1}} -b^g_k) ]$$

### 状態の誤差
$$ x = \hat{x} + \delta{x} $$
* $\hat{x}$ 真値
* $\delta{x}$ 誤差
$$ x_{k+1} + \delta{x}_{k+1} = f(x_{k}, u_{k}) $$
$$ \hat{x}_{k+1} + \delta{x}_{k+1} = f(\hat{x}_{k}, \hat{u}_{k}) + F \delta{x_{k}} + G n_{k}$$
* $\hat{x}_{k}$ kの状態の真値
* $\hat{u}_{k}$ kのinput
$$ F = \frac{\partial f(x_{k}, u_{k})}{\partial x_{k}} $$
$$ G = \frac{\partial f(x_{k}, u_{k})}{\partial n_{k}} $$


$$ F = \left[
\begin{matrix} 
0 & f_{12} &  I \delta t & -\frac{1}{2}(q_{b_{i} b_{k}} + q_{b_{i} b_{k+1}}) \delta t^2 & f_{15}
\\ 
0 & f_{22} & 0 & 0 & I \delta t
\\ 
0 & f_{32} & I & -\frac{1}{2}(q_{b_{i} b_{k}} + q_{b_{i} b_{k+1}}) \delta t  & f_{35}
\\ 
0 & 0 & 0 & I & 0
\\ 
0 & 0 & 0 & 0 & I
 \end{matrix}
\right] $$

$$ f_{22} = \frac{\partial \delta q_{k+1}}{\partial \delta q_{k}} $$
$$q_{b_i b_{k+1}} \otimes \left[
\begin{matrix} 1 \\ \delta q_{k+1} /2 \end{matrix}
\right] 
= q_{b_i b_{k}} \otimes \left[
\begin{matrix} 1 \\ \delta q_{k} /2 \end{matrix}
\right]  \otimes \left[
\begin{matrix} 1 \\ \omega \delta t /2 \end{matrix}
\right] $$

$$ \left[
\begin{matrix} 1 \\ \delta q_{k+1} /2 \end{matrix}
\right] 
= q_{b_{k+1} b_{i}} q_{b_i b_{k}} \otimes \left[
\begin{matrix} 1 \\ \delta q_{k} /2 \end{matrix}
\right]  \otimes \left[
\begin{matrix} 1 \\ \omega \delta t /2 \end{matrix}
\right] $$

$$ \left[
\begin{matrix} 1 \\ \delta q_{k+1} /2 \end{matrix}
\right] 
= q_{b_{k+1} b_{k}} \otimes \left[
\begin{matrix} 1 \\ \delta q_{k} /2 \end{matrix}
\right]  \otimes \left[
\begin{matrix} 1 \\ \omega \delta t /2 \end{matrix}
\right] $$

$$ \left[
\begin{matrix} 1 \\ \delta q_{k+1} /2 \end{matrix}
\right] 
= \left[
\begin{matrix} 1 \\ -\omega \delta t /2 \end{matrix}
\right]  \otimes \left[
\begin{matrix} 1 \\ \delta q_{k} /2 \end{matrix}
\right]  \otimes \left[
\begin{matrix} 1 \\ \omega \delta t /2 \end{matrix}
\right] $$

$$ 
= \left[
\begin{matrix} 1 \\ R \delta q_{b_k} /2 \end{matrix}
\right]$$

$$ \delta q_{k+1} /2 = R \delta q_{b_k} /2 $$
$$ q_{k+1} = (I - [\omega \delta t]\times) \delta q_{b_k}  $$
$$ f_{22} = I - [\omega \delta t]\times  $$